# Carbon Use Efficiency

Resources:
* Models: http://bigg.ucsd.edu/models/

In [ ]:
!pip install cobra

     |████████████████████████████████| 2.4MB 9.5MB/s 
     |████████████████████████████████| 133kB 38.4MB/s 
     |████████████████████████████████| 10.1MB 24.7MB/s 
     |████████████████████████████████| 6.2MB 30.6MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 153kB 56.7MB/s 
     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 2.1MB 27.8MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 552kB 42.9MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 


In [ ]:
import requests, cobra, tempfile

# Functions

In [ ]:
def loadModelURL(url):
  """Load SMBL cobra model from a URL"""
  with tempfile.NamedTemporaryFile('w') as tf:
    handle = requests.get(url)
    tf.write(handle.text)
    model = cobra.io.read_sbml_model(tf.name)
    return model

# Load

In [ ]:
"""Load model"""
url = 'http://bigg.ucsd.edu/static/models/e_coli_core.xml'
model = loadModelURL(url)
model

Name,e_coli_core
Memory address,0x07efd219fde10
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [31]:
res = model.optimize()
